# Mathematical Engineering - Financial Engineering, FY 2024-2025
# Risk Management - Exercise 5: Extended Vasiceck model Monte Carlo simulation for CCR estimation

In [1]:
# Importing the libraries
import numpy as np
import pandas as pd


from utilities.ex1_utilities import (
    business_date_offset,
    date_series,
    bootstrap,
    SwapType,
    swap_par_rate,
    year_frac_30_360,
    from_discount_factors_to_zero_rates,
    get_discount_factor_by_zero_rates_linear_interp,
    year_frac_act_x
)
from utilities.ex5_utilities import (
    affine_trick,
)

In [2]:
filename = "MktData_CurveBootstrap.xls"
dates, rates , discount_factors  = bootstrap(filename,0)

today = dates[0]

In [3]:
# Parameters
swap_type = SwapType.PAYER
maturity = 8  # Maturity in years
fixed_freq = 4  # Number of times per year

mean_reversion_speed = 0.0195
sigma = 0.0086

expiry = business_date_offset(today, year_offset=maturity)
fixed_leg_payment_dates = date_series(today, expiry, fixed_freq)

swap_rate = swap_par_rate(today,fixed_leg_payment_dates, discount_factors,None,dates)
print(f"Swap rate: {swap_rate:.4%}")

Swap rate: 2.7886%


In [4]:
# Monte Carlo Simulations
simulations_num = 250_000
simulation_grid = fixed_leg_payment_dates[1:]

alpha = 0.95  # PFE confidence level

expected_exposure = pd.DataFrame(
    data=0.0, index=simulation_grid, columns=["no_collateral", "collateral"]
)
potential_future_exposure = pd.DataFrame(
    data=0.0, index=simulation_grid, columns=["no_collateral", "collateral"]
)



In [5]:
# Compute IRS cash flows
new_discount = [get_discount_factor_by_zero_rates_linear_interp(today, d, dates, discount_factors) for d in fixed_leg_payment_dates[1:]]
irs_cash_flows = pd.Series(index=fixed_leg_payment_dates[1:])
for i in range(1, len(fixed_leg_payment_dates)):
    delta = year_frac_30_360(fixed_leg_payment_dates[i-1], fixed_leg_payment_dates[i])
    irs_cash_flows[i-1] = delta * swap_rate

print(f"IRS cash flows:\n{irs_cash_flows}\n")


IRS cash flows:
2023-05-02    0.006971
2023-08-02    0.006971
2023-11-02    0.006971
2024-02-02    0.006971
2024-05-02    0.006971
2024-08-02    0.006971
2024-11-04    0.007126
2025-02-03    0.006894
2025-05-02    0.006894
2025-08-04    0.007126
2025-11-03    0.006894
2026-02-02    0.006894
2026-05-04    0.007126
2026-08-03    0.006894
2026-11-02    0.006894
2027-02-02    0.006971
2027-05-03    0.007049
2027-08-02    0.006894
2027-11-02    0.006971
2028-02-02    0.006971
2028-05-02    0.006971
2028-08-02    0.006971
2028-11-02    0.006971
2029-02-02    0.006971
2029-05-02    0.006971
2029-08-02    0.006971
2029-11-02    0.006971
2030-02-04    0.007126
2030-05-02    0.006817
2030-08-02    0.006971
2030-11-04    0.007126
2031-02-03    0.006894
dtype: float64



C:\Users\Salvo\AppData\Local\Temp\ipykernel_17224\3716204272.py:6: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  irs_cash_flows[i-1] = delta * swap_rate


In [6]:
Z = np.random.normal(size=(simulations_num, len(simulation_grid)))

# Initialize collateral tracking
collateral = np.zeros(simulations_num)  # Track collateral separately for each simulation
last_collateral_update = today  # Track last collateral update date

for count, sim_date in enumerate(simulation_grid):
    if count == 0:
        prev_sim_date = today
        x_t = np.zeros((1,simulations_num))
    else:
        prev_sim_date = simulation_grid[count - 1]

    # Simulate the short rate x_t
    delta_t = year_frac_act_x(prev_sim_date, sim_date, 365)
    x_t = x_t * np.exp(-mean_reversion_speed * delta_t) + \
          sigma * np.sqrt((1 - np.exp(-2 * mean_reversion_speed * delta_t)) /
               (2 * mean_reversion_speed)) * Z[:, count]

    A, C = affine_trick(
        sim_date,
        today,
        simulation_grid,
        mean_reversion_speed,
        sigma,
        new_discount,
    )

    # Simulated MtM
    MtM = (-1 if swap_type == SwapType.PAYER else 1) * (
        np.sum(
            np.exp(
                np.log(irs_cash_flows * A).values.reshape(-1, 1)
                @ np.ones((1, simulations_num))
                - C.values.reshape(-1, 1) @ x_t.reshape(1, -1)
            ),
            axis=0,
        )
        - A.iloc[-1]*np.exp(-x_t*C.iloc[-1])
        - (-1 if sim_date < simulation_grid[-1] else 0)
    )
    MtMs = MtM[:, count]
    # Annual collateral update check
    if sim_date.year > last_collateral_update.year:  # Annual update
        collateral = MtM
        last_collateral_update = sim_date

    # Calculate exposures
    credit_exposure = np.maximum(MtM, 0)
    probability = np.mean(MtM > 0)
    collateral_exposure = np.maximum(MtM - collateral, 0)

    # Store results
    expected_exposure.loc[count, "no_collateral"] = np.mean(credit_exposure)
    expected_exposure.loc[count, "collateral"] = np.mean(collateral_exposure)

    potential_future_exposure.loc[count, "no_collateral"] = np.quantile(credit_exposure, alpha)
    potential_future_exposure.loc[count, "collateral"] = np.quantile(collateral_exposure, alpha)

# Calculate summary metrics
expected_positive_exposure = {
    "no_collateral": np.mean(expected_exposure["no_collateral"]),
    "collateral": np.mean(expected_exposure["collateral"])
}

peak_pfe = {
    "no_collateral": np.max(potential_future_exposure["no_collateral"]),
    "collateral": np.max(potential_future_exposure["collateral"])
}

In [7]:
# Print results
print("\n===== Results with Annual Collateral =====")
print(f"EPE without collateral: {expected_positive_exposure['no_collateral']:.6f}")
print(f"EPE with annual collateral: {expected_positive_exposure['collateral']:.6f} (reduction: {100*(1-expected_positive_exposure['collateral']/expected_positive_exposure['no_collateral']):.1f}%)")
print(f"\nPeak PFE without collateral: {peak_pfe['no_collateral']:.6f}")
print(f"Peak PFE with annual collateral: {peak_pfe['collateral']:.6f} (reduction: {100*(1-peak_pfe['collateral']/peak_pfe['no_collateral']):.1f}%)")


===== Results with Annual Collateral =====
EPE without collateral: 0.012144
EPE with annual collateral: 0.005095 (reduction: 58.0%)

Peak PFE without collateral: 0.776836
Peak PFE with annual collateral: 0.109501 (reduction: 85.9%)
